# Exercise with Natural Language Processing

For todays exersice we will be doing two things.  The first is to build the same model with the same data that we did in the lecture, the second will be to build a new model with new data. 

## PART 1: 
- 20 Newsgroups Corpus


## PART 2:
- Republican vs Democrat Tweet Classifier

In [1]:
# Import pandas for data handling
import pandas as pd

# NLTK is our Natural-Language-Took-Kit
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Libraries for helping us with strings
import string
# Regular Expression Library
import re

# Import our text vectorizers
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


# Import our classifiers
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier


# Import some ML helper function
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report


# Import our metrics to evaluate our model
from sklearn import metrics
from sklearn.metrics import classification_report


# Library for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# You may need to download these from nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/remilaurence/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/remilaurence/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/remilaurence/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Load and display data.
1. Load the 20-newsgroups.csv data into a dataframe.
1. Print the shape
1. Inspect / remove nulls and duplicates
1. Find class balances, print out how many of each topic_category there are.

In [2]:
# 1. Load the 20-newsgroups.csv data into a dataframe.
# 2. Print the shape
df = pd.read_csv('data/20-newsgroups.csv')
df.shape

(11314, 4)

In [3]:
# 3. Inspect / remove nulls and duplicates
print(df.isnull().sum())
print(df.duplicated().sum())

id                0
message           0
topic             0
topic_category    0
dtype: int64
0


In [4]:
# 4. Find class balances, print out how many of each topic_category there are.
df.topic_category.value_counts()

rec.sport.hockey            600
soc.religion.christian      599
rec.motorcycles             598
rec.sport.baseball          597
sci.crypt                   595
rec.autos                   594
sci.med                     594
sci.space                   593
comp.windows.x              593
sci.electronics             591
comp.os.ms-windows.misc     591
comp.sys.ibm.pc.hardware    590
misc.forsale                585
comp.graphics               584
comp.sys.mac.hardware       578
talk.politics.mideast       564
talk.politics.guns          546
alt.atheism                 480
talk.politics.misc          465
talk.religion.misc          377
Name: topic_category, dtype: int64

# Text Pre-Processing 
(aka Feature engineering)
1. Make a function that makes all text lowercase.
    * Do a sanity check by feeding in a test sentence into the function. 
    
    
2. Make a function that removes all punctuation. 
    * Do a sanity check by feeding in a test sentence into the function. 
    
    
3. Make a function that removes all stopwords.
    * Do a sanity check by feeding in a test sentence into the function. 
    
    
4. EXTRA CREDIT (This step only): Make a function that stemms all words. 


5. Mandatory: Make a pipeline function that applys all the text processing functions you just built.
    * Do a sanity check by feeding in a test sentence into the pipeline. 
    
    
    
6. Mandatory: Use `df['message_clean'] = df[column].apply(???)` and apply the text pipeline to your text data column. 

In [5]:
# 1. Make a function that makes all text lowercase.
def make_lower(a_string):
    return a_string.lower()

test_string = 'This is A SENTENCE with LOTS OF CAPS.'
print(make_lower(test_string))


this is a sentence with lots of caps.


In [6]:
# 2. Make a function that removes all punctuation. 
def remove_punctuation(a_string):
    a_string = re.sub(r'[^\w\s]','',a_string)
    return a_string

test_string = 'This is a sentence! 50 With lots of punctuation??? & other #things.'
print(remove_punctuation(test_string))

This is a sentence 50 With lots of punctuation  other things


In [7]:
# 3. Make a function that removes all stopwords.
def remove_stopwords(a_string):
    words = word_tokenize(a_string)
    valid_words = []
    for word in words:
        if word not in stopwords:
            valid_words.append(word)
    a_string = ' '.join(valid_words)
    return a_string
    
test_string = 'This is a sentence! With some different stopwords i have added in here.'
print(remove_stopwords(test_string))

This sentence ! With different stopwords added .


In [8]:
# 4. EXTRA CREDIT: Make a function that stemms all words. 
def stem_words(a_string):
    porter = PorterStemmer()
    words = word_tokenize(a_string)
    valid_words = []
    for word in words:
        stemmed_word = porter.stem(word)
        valid_words.append(stemmed_word)
    a_string = ' '.join(valid_words)
    return a_string


test_string = 'I played and started playing with players and we all love to play with plays'
print(stem_words(test_string))


I play and start play with player and we all love to play with play


In [9]:
# 5. MANDATORY: Make a pipeline function that applys all the text processing functions you just built.
def text_pipeline(a_string):
    a_string = make_lower(a_string)
    a_string = remove_punctuation(a_string)
    a_string = remove_stopwords(a_string)
    a_string = stem_words(a_string)
    return a_string

test_string = 'I played and started playing with players and we all love to play with plays'
print(text_pipeline(test_string))

play start play player love play play


In [10]:
# 6. Mandatory: Use `df[column].apply(???)` and apply the text pipeline to your text data column. 
df['message_clean'] = df['message'].apply(text_pipeline)

# Text Vectorization

1. Define your `X` and `y` data. 


2. Initialize a vectorizer (you can use TFIDF or BOW, it is your choice).
    * Do you want to use n-grams..?


3. Fit your vectorizer using your X data.
    * Remember, this process happens IN PLACE.


4. Transform your X data using your fitted vectorizer. 
    * `X = vectorizer.???`



5. Print the shape of your X.  How many features (aka columns) do you have?

In [11]:
# 1. Define your `X` and `y` data. 
X = df['message_clean'].values
y = df['topic_category'].values

In [12]:
# 2. Initialize a vectorizer (you can use TFIDF or BOW, it is your choice).
vect = TfidfVectorizer()
vect.fit(X)
X = vect.transform(X)

In [13]:
# 3. Fit your vectorizer using your X data


In [14]:
# 4. Transform your X data using your fitted vectorizer. 



In [15]:
# 5. Print the shape of your X.  How many features (aka columns) do you have?

print(X.shape)

(11314, 119708)


# Split your data into Training and Testing data. 

In [16]:
# Split our data into testing and training like always. 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state=10)

___
# Build and Train Model
Use Multinomial Naive Bayes to classify these documents. 

1. Initalize an empty model. 
2. Fit the model with our training data.


Experiment with different alphas.  Use the alpha gives you the best result.

EXTRA CREDIT:  Use grid search to programmatically do this for you. 

In [17]:
# 1. Initalize an empty model. 

model = MultinomialNB()


In [18]:
# Fit our model with our training data.

model.fit(X_train, y_train)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

# Evaluate the model.

1. Make new predicitions using our test data. 
2. Print the accuracy of the model. 
3. Print the confusion matrix of our predictions. 
4. Using `classification_report` print the evaluation results for all the classes. 



In [19]:
# 1. Make new predictions of our testing data. 

y_pred = model.predict(X_test)


In [20]:
# 2. Print the accuracy of the model. 
accuracy = model.score(X_test, y_test)

print("Model Accuracy: %f" % accuracy)

Model Accuracy: 0.857269


In [21]:
# 3. Plot the confusion matrix of our predictions
#I was getting an error trying to import it above so just tried to redo it down here
from sklearn.metrics import plot_confusion_matrix

fig, ax = plt.subplots(figsize=(21, 21))

disp = plot_confusion_matrix(model, X_test, y_test,
                             display_labels=model.classes_,
                             cmap=plt.cm.Blues, ax=ax)
plt.xticks(rotation=90)
disp

ImportError: cannot import name 'plot_confusion_matrix' from 'sklearn.metrics' (/Users/remilaurence/anaconda3/lib/python3.7/site-packages/sklearn/metrics/__init__.py)

In [22]:
# 4. Using `classification_report` print the evaluation results for all the classes. 

print(classification_report(y_test, y_pred, target_names=model.classes_))

                          precision    recall  f1-score   support

             alt.atheism       0.93      0.81      0.86        94
           comp.graphics       0.89      0.77      0.83       120
 comp.os.ms-windows.misc       0.77      0.87      0.82       109
comp.sys.ibm.pc.hardware       0.71      0.81      0.76       113
   comp.sys.mac.hardware       0.88      0.82      0.85       118
          comp.windows.x       0.89      0.87      0.88       110
            misc.forsale       0.88      0.66      0.75       118
               rec.autos       0.88      0.91      0.89       117
         rec.motorcycles       0.95      0.96      0.96       111
      rec.sport.baseball       0.96      0.96      0.96       116
        rec.sport.hockey       0.92      0.97      0.94       112
               sci.crypt       0.78      0.97      0.86       131
         sci.electronics       0.89      0.72      0.80       134
                 sci.med       0.97      0.91      0.94       122
         

# Manual predicition
Write a new sentence that you think will be classified as talk.politics.guns. 
1. Apply the text pipeline to your sentence
2. Transform your cleaned text using the `X = vectorizer.transform([your_text])`
    * Note, the `transform` function accepts a list and not a individual string.
3. Use the model to predict your new `X`. 
4. Print the prediction

In [59]:
my_sentence = "I enjoy my second ammendment right to bear arms because I like to hunt on the weekends and go to the shooting range"

# 1. Apply the text pipeline to your sentence
my_sentence = text_pipeline(my_sentence)

# 2. Transform your cleaned text using the `X = vectorizer.transform([your_text])`\

X = vect.transform([my_sentence])

# 3. Use the model to predict your new `X`. 
y_pred = model.predict(X)

# 4. Print the prediction
print(y_pred)

['Republican']


___
# PART 2: Twitter Data
This part of the exercise is un-guided on purpose.  

Using the `dem-vs-rep-tweets.csv` build a classifier to determine if a tweet was written by a democrat or republican. 

Can you get an f1-score higher than %82

In [41]:
# 1. Load the 20-newsgroups.csv data into a dataframe.
# 2. Print the shape
df = pd.read_csv('data/dem-vs-rep-tweets.csv')
print(df.shape)
print(df.isnull().sum())

(86460, 3)
Party     0
Handle    0
Tweet     0
dtype: int64


In [42]:
df.duplicated().sum()

57

In [47]:
df = df.drop_duplicates()
df.duplicated().sum()

0

In [49]:
df['clean_tweet'] = df['Tweet'].apply(text_pipeline)

In [51]:
X = df['clean_tweet']
y = df['Party']

In [52]:
vect = TfidfVectorizer()
vect.fit(X)
X = vect.transform(X)

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state=10)

In [93]:
model = MultinomialNB(alpha=2.3)

In [94]:
model.fit(X_train, y_train)

MultinomialNB(alpha=2.3, class_prior=None, fit_prior=True)

In [95]:
y_pred = model.predict(X_test)

In [96]:
accuracy = model.score(X_test, y_test)

print("Model Accuracy: %f" % accuracy)

print(classification_report(y_test, y_pred, target_names=model.classes_))

Model Accuracy: 0.787917
              precision    recall  f1-score   support

    Democrat       0.82      0.72      0.77      8394
  Republican       0.76      0.85      0.81      8887

   micro avg       0.79      0.79      0.79     17281
   macro avg       0.79      0.79      0.79     17281
weighted avg       0.79      0.79      0.79     17281



In [97]:
rfmodel = RandomForestClassifier(n_estimators=150, max_depth=20)
rfmodel.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [88]:
y_pred = rfmodel.predict(X_test)

accuracy =  rfmodel.score(X_test, y_test)

print("Model Accuracy: %f" % accuracy)

print(classification_report(y_test, y_pred, target_names=rfmodel.classes_))

Model Accuracy: 0.566981
              precision    recall  f1-score   support

    Democrat       0.92      0.12      0.21      8394
  Republican       0.54      0.99      0.70      8887

   micro avg       0.57      0.57      0.57     17281
   macro avg       0.73      0.55      0.46     17281
weighted avg       0.72      0.57      0.46     17281



In [100]:
new_tweet = 'Ronald Reagan is the best president of all time. He is great because of tax breaks'
new_tweet = text_pipeline(new_tweet)
X = vect.transform([new_tweet])
print(model.predict(X))


['Republican']
